In [1]:
import pandas as pd
import os

rotas=pd.read_parquet("dados/rotas.parquet")
viagens=pd.read_parquet("dados/viagens.parquet")
passageiros=pd.read_parquet("dados/passageiros_unificado.parquet")
passageiros=passageiros[['data', 'linha', 'passageiros_total']].rename(columns={"linha":"route_id"})
passageiros=passageiros.merge(rotas[['route_long_name','route_id']],on='route_id')
passageiros['ano']=passageiros.data.dt.year
passageiros['mes']=passageiros.data.dt.month

In [3]:
print(rotas.columns,'\n' ,viagens.columns)

Index(['route_id', 'agency_id', 'route_short_name', 'route_long_name',
       'route_type', 'route_color', 'route_text_color'],
      dtype='object') 
 Index(['route_id', 'service_id', 'trip_id', 'trip_headsign', 'direction_id',
       'shape_id', 'stop_id', 'stop_sequence', 'stop_name', 'stop_desc',
       'stop_lat', 'stop_lon'],
      dtype='object')


In [72]:
print("Rotas:",rotas.columns)
print("Viagens:",viagens.columns)
print("Passageiros:",passageiros.columns)

Rotas: Index(['route_id', 'agency_id', 'route_short_name', 'route_long_name',
       'route_type', 'route_color', 'route_text_color'],
      dtype='object')
Viagens: Index(['route_id', 'service_id', 'trip_id', 'trip_headsign', 'direction_id',
       'shape_id', 'stop_id', 'stop_sequence', 'stop_name', 'stop_desc',
       'stop_lat', 'stop_lon'],
      dtype='object')
Passageiros: Index(['data', 'route_id', 'passageiros_total', 'route_long_name', 'ano',
       'mes'],
      dtype='object')


Análise estatística inicial

In [65]:
# top 10 nos últimos 10 anos
passageiros.sort_values("passageiros_total",ascending=False).head(10)

,data,route_id,passageiros_total,route_long_name,ano,mes
1028258,2019-06-14,431010,58124.0,E.t. Itaquera - Term. Pq. D. Pedro Ii,2019,6
1030719,2018-01-18,431010,54309.0,E.t. Itaquera - Term. Pq. D. Pedro Ii,2018,1
1028812,2018-10-21,431010,52612.0,E.t. Itaquera - Term. Pq. D. Pedro Ii,2018,10
1028688,2021-05-19,431010,50157.0,E.t. Itaquera - Term. Pq. D. Pedro Ii,2021,5
1029044,2023-03-23,431010,48695.0,E.t. Itaquera - Term. Pq. D. Pedro Ii,2023,3
2865776,2018-10-21,220210,47974.0,Jd. Das Oliveiras - Cptm Guaianases,2018,10
1030352,2017-02-07,431010,47532.0,E.t. Itaquera - Term. Pq. D. Pedro Ii,2017,2
1029860,2015-08-17,431010,46835.0,E.t. Itaquera - Term. Pq. D. Pedro Ii,2015,8
1029861,2015-08-18,431010,46810.0,E.t. Itaquera - Term. Pq. D. Pedro Ii,2015,8
2131761,2018-10-21,675K10,46682.0,Term. Jd. âNgela - Metrô Sta. Cruz,2018,10


In [57]:
# top 5 maiores médias nos últimos 10 anos
passageiros.groupby(['route_id','route_long_name']).mean()['passageiros_total'].reset_index().sort_values("passageiros_total",ascending=False).head(5)

,route_id,route_long_name,passageiros_total
463,431010,E.t. Itaquera - Term. Pq. D. Pedro Ii,24477.349918
793,675K10,Term. Jd. âNgela - Metrô Sta. Cruz,22472.211620
159,220210,Jd. Das Oliveiras - Cptm Guaianases,21481.771981
728,607C10,Jd. Miriam - Itaim Bibi,21275.563081
830,691310,Term. Varginha - Term. Bandeira,21131.116905


In [62]:
# top 5 maiores médias por ano
passageiros.groupby(['route_id','route_long_name','ano']).mean()['passageiros_total'].reset_index().sort_values("passageiros_total",ascending=False).head(5)

,route_id,route_long_name,ano,passageiros_total
7711,675K10,Term. Jd. âNgela - Metrô Sta. Cruz,2016,32558.293769
7710,675K10,Term. Jd. âNgela - Metrô Sta. Cruz,2015,31487.969799
7712,675K10,Term. Jd. âNgela - Metrô Sta. Cruz,2017,31413.395887
4482,431010,E.t. Itaquera - Term. Pq. D. Pedro Ii,2016,30738.508876
4481,431010,E.t. Itaquera - Term. Pq. D. Pedro Ii,2015,30587.488294


In [69]:
# top 5 maiores médias em anos pandêmicos
passageiros.loc[passageiros.ano.isin([2020,2021,2022])].groupby(['route_id','route_long_name']).mean()['passageiros_total'].reset_index().sort_values("passageiros_total",ascending=False).head(5)

,route_id,route_long_name,passageiros_total
456,431010,E.t. Itaquera - Term. Pq. D. Pedro Ii,21500.428209
636,600010,Term. Parelheiros - Term. Sto. Amaro,19679.637774
157,220210,Jd. Das Oliveiras - Cptm Guaianases,19524.613879
718,607C10,Jd. Miriam - Itaim Bibi,19331.476190
461,431310,Term. Cid. Tiradentes - Term. Pq. D. Pedro Ii,18637.985650


In [70]:
# top 5 maiores médias em anos recentes
passageiros.loc[passageiros.ano.isin([2023,2024,2025])].groupby(['route_id','route_long_name']).mean()['passageiros_total'].reset_index().sort_values("passageiros_total",ascending=False).head(5)

,route_id,route_long_name,passageiros_total
728,607C10,Jd. Miriam - Itaim Bibi,26095.710660
643,600010,Term. Parelheiros - Term. Sto. Amaro,21964.986644
463,431010,E.t. Itaquera - Term. Pq. D. Pedro Ii,19163.449843
468,431310,Term. Cid. Tiradentes - Term. Pq. D. Pedro Ii,19065.104000
793,675K10,Term. Jd. âNgela - Metrô Sta. Cruz,18295.121643


In [71]:
# top 5 maiores médias em anos anteriores
passageiros.loc[passageiros.ano.isin([2015,2016,2027,2018])].groupby(['route_id','route_long_name']).mean()['passageiros_total'].reset_index().sort_values("passageiros_total",ascending=False).head(5)

,route_id,route_long_name,passageiros_total
448,431010,E.t. Itaquera - Term. Pq. D. Pedro Ii,28508.841360
772,675K10,Term. Jd. âNgela - Metrô Sta. Cruz,26899.279349
809,691310,Term. Varginha - Term. Bandeira,26294.827747
154,220210,Jd. Das Oliveiras - Cptm Guaianases,24251.595177
282,306410,Cid. Tiradentes - Cptm Guaianases,21352.830769
